<h1> <center> Exploiting Sentinel-1 imagery time series to detect grasslands in northern Brazil tropical plains</center> </h1>
<center> Arian Ferreira Carneiro </center>
<center>Willian Vieira de Oliveira </center>

[Dicas Sidnei] Para justificar o trabalho:

- buscar imagens ópticas
- mostrar as limitações dessas imagens e a importância do uso de imagens de radar na região

<b>Websites that might be useful: </b>

https://gee-python-api.readthedocs.io/en/latest/ee.html

   - Converting from javascript to python api
   
    https://gis.stackexchange.com/questions/336080/converting-map-from-javascript-api-to-python-api
    
    https://github.com/GreenInfo-Network/earthengine-prototyping/issues/6
    
    
   - GEE data in Pandas
    
    https://mygeoblog.com/2017/01/13/your-gee-data-in-pandas/
    
    https://mygeoblog.com/2017/10/06/from-gee-to-numpy-to-geotiff/
    
    
   - Others:
    
    https://www.linkedin.com/pulse/cloud-computing-land-cover-classification-part-1-how-jo%C3%A3o-otavio/
    

## Import required packages

In [1]:
# Import the Earth Engine Python Package
import ee

"""
In addition to syntactic differences between JavaScript and Python (e.g. declare a user function in Python with 
def instead of function), logical methods are capitalized in Python to avoid reserved words: And(), Or(), Not() 
instead of and(), or() not(). Map output is currently not supported.
"""

# Initialize the Earth Engine object, using the authentication credentials
ee.Initialize()

In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from osgeo import gdal
#from osgeo import osr
#import time


## Input parameters

In [3]:
# ÁREA DE ESTUDO
#geometry = ee.Geometry.Polygon([[[-55.00, -3.48], [-55.00, -3.50], [-54.98, -3.50], [-54.98, -3.48]]]);
geometry = ee.Geometry.Polygon([[[-54.862897883732444, -2.711477926102956], [-54.862897883732444, -2.9117369535738455], [-54.549787532169944, -2.9117369535738455], [-54.549787532169944, -2.711477926102956]]], None, False)

In [4]:
# Períodos de análise: Estações do ano
  # Primavera
data_1 = "2017-09-21"; 
data_2 = "2017-12-20";
primavera = ee.Filter.date(data_1, data_2);
  # Verão
data_3 = "2017-12-21"; 
data_4 = "2018-03-20";
verao = ee.Filter.date(data_3, data_4);
  # Outono
data_5 = "2018-03-21"; 
data_6 = "2018-06-21";
outono = ee.Filter.date(data_5, data_6);
  # Inverno
data_7 = "2018-06-22"; 
data_8 = "2018-09-21";
inverno = ee.Filter.date(data_7, data_8);

# Período completo!
periodo = ee.Filter.date(data_1, data_8);

## Functions

In [5]:
# FUNCTIONS
def toClip(img):
    return ee.Image(img).clip(geometry);


def QEY(imgCol,N):
    """  
    Função para filtrar uma coleção seguindo a metodologia de Quegan&Yu, 2001
    Importante: a coleção deve estar em números naturais. N=tamanho do filtro espacial
    This function will return a filtered collection 
    The filter is the proposed by Quegan&Yu (2011),
    Warning: will only work on single scene time series on natural. No mosaics (for now), no DB
    """
    
    boxcar = ee.Kernel.square(N, 'pixels', False);
    
    imgColMedian = imgCol.map(lambda img: img.convolve(boxcar))

    correctionFactorCol=imgCol.map(lambda img: img.divide(img.convolve(boxcar)))

    numberofsamples=imgCol.count()
    correctionFactor=correctionFactorCol.sum().divide(numberofsamples)
    
    return imgColMedian.map(lambda img: img.multiply(correctionFactor).copyProperties(img,['system:time_start','sliceNumber']))


def extractDates(col): # dates are originally in milliseconds. This function converts them to 'YYYY-MM-dd'
    def formatDate(date):
        return ee.Date(date).format('YYYY-MM-dd')
    
    datesList=ee.List(col.aggregate_array('system:time_start').getInfo())
    datesListFmt=datesList.map(formatDate);
    return datesListFmt;


def renameDates(img, datesListFmt):
    band_names = img.bandNames()
    new_names = datesListFmt
    return img.select(band_names).rename(new_names);


def RGI(collec):
    rgi = collec.normalizedDifference(['VV', 'VH']).rename('RGI');
    return rgi.copyProperties(collec,['system:time_start','sliceNumber']);


def NL(collec):
    nl = collec.expression('((VV*VH)/(VV+VH))', {
    'VV': collec.select('VV'),
    'VH': collec.select('VH')}).rename('NL');
    return nl.copyProperties(collec,['system:time_start','sliceNumber']);


def Ratio(collec):
    ratio = collec.expression('VH/VV', {
    'VV': collec.select('VV'),
    'VH': collec.select('VH')}).rename('ratio');
    return ratio.copyProperties(collec,['system:time_start','sliceNumber']);

## Procedures

In [6]:
# ------------------------- DATA CUBE -------------------------

# Chamando a coleção S1A_GRD em linear (FLOAT)
s1a = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).filter(ee.Filter.eq('platform_number', 'A')).filterBounds(geometry).filter(periodo).map(toClip)

In [7]:
# Criando collections com as cenas de mesma data mosaicadas.
s1p=QEY(s1a.filter(primavera), 5);
s1v=QEY(s1a.filter(verao), 5);
s1o=QEY(s1a.filter(outono), 5);
s1i=QEY(s1a.filter(inverno), 5);

s1a_filt = s1p.merge(s1v).merge(s1o).merge(s1i);

vv = s1a_filt.select('VV')
vh = s1a_filt.select('VH')
ratio = s1a_filt.map(Ratio)
RGI = s1a_filt.map(RGI)
NL = s1a_filt.map(NL)

In [8]:
vv.size().getInfo()

31

In [9]:
vv_cube = vv.toBands()

In [10]:
#vv_cube.getInfo()
vv_cube.bandNames().getInfo()

['1_1_1_S1A_IW_GRDH_1SDV_20170922T092246_20170922T092311_018486_01F234_89ED_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171004T092247_20171004T092312_018661_01F78D_CFB2_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171016T092247_20171016T092312_018836_01FCE5_6E12_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171028T092247_20171028T092312_019011_020236_8AD1_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171109T092246_20171109T092311_019186_02079B_A15D_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171121T092246_20171121T092311_019361_020D21_A36E_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171203T092246_20171203T092311_019536_021298_BC9B_VV',
 '1_1_1_S1A_IW_GRDH_1SDV_20171215T092246_20171215T092311_019711_02180C_6CEB_VV',
 '1_1_2_S1A_IW_GRDH_1SDV_20171227T092245_20171227T092310_019886_021D7B_8702_VV',
 '1_1_2_S1A_IW_GRDH_1SDV_20180108T092245_20180108T092310_020061_0222FA_1554_VV',
 '1_1_2_S1A_IW_GRDH_1SDV_20180120T092244_20180120T092309_020236_022888_280B_VV',
 '1_1_2_S1A_IW_GRDH_1SDV_20180201T092244_20180201T092309_020411_022E18_A24A_VV',
 '1_1_2_S1A_IW_GRDH_1SDV_201

In [11]:
datesListFmt = extractDates(vv)
#datesListFmt.getInfo()

In [12]:
vv_cube = renameDates(vv_cube, datesListFmt)
vv_cube.bandNames().getInfo()

['2017-09-22',
 '2017-10-04',
 '2017-10-16',
 '2017-10-28',
 '2017-11-09',
 '2017-11-21',
 '2017-12-03',
 '2017-12-15',
 '2017-12-27',
 '2018-01-08',
 '2018-01-20',
 '2018-02-01',
 '2018-02-13',
 '2018-02-25',
 '2018-03-09',
 '2018-03-21',
 '2018-04-02',
 '2018-04-14',
 '2018-04-26',
 '2018-05-08',
 '2018-05-20',
 '2018-06-01',
 '2018-06-13',
 '2018-06-25',
 '2018-07-07',
 '2018-07-19',
 '2018-07-31',
 '2018-08-12',
 '2018-08-24',
 '2018-09-05',
 '2018-09-17']

In [50]:
# get the lat, lon
latlon = ee.Image.pixelLonLat().addBands(vv_cube)

# apply reducer to list
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry= geometry,
    scale=30,
    maxPixels= 1e9);

In [51]:
df_list = []

#lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
#lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))

#for band in vv_cube.bandNames().getInfo()[0:2]:
for band in vv_cube.bandNames().getInfo():
    print("Processing: ", band)
    data = np.array(ee.Array(latlon.get(band)).getInfo())
    df = pd.DataFrame(data, columns=[band])
    df_list.append(df)

Processing:  2017-09-22


EEException: Computed value is too large.

In [43]:
df_vv_cube = pd.concat(df_list, axis=1)
df_vv_cube

,2017-09-22,2017-10-04,2017-10-16,2017-10-28,2017-11-09,2017-11-21,2017-12-03,2017-12-15,2017-12-27,2018-01-08,...,2018-06-01,2018-06-13,2018-06-25,2018-07-07,2018-07-19,2018-07-31,2018-08-12,2018-08-24,2018-09-05,2018-09-17
0,0.052760,0.043937,0.044622,0.056292,0.053256,0.051523,0.048513,0.106305,0.141518,0.111147,...,0.118521,0.108831,0.139649,0.116031,0.128355,0.121563,0.100629,0.088314,0.111552,0.099742
1,0.049352,0.043025,0.043376,0.050918,0.048498,0.048214,0.045763,0.089475,0.135280,0.110368,...,0.128506,0.118000,0.113688,0.093170,0.105300,0.101248,0.085914,0.075280,0.093589,0.086390
2,0.057120,0.049148,0.049309,0.057543,0.054745,0.053417,0.050916,0.092713,0.130687,0.108378,...,0.121545,0.116415,0.116606,0.097495,0.107735,0.106969,0.091642,0.081250,0.098986,0.092297
3,0.106393,0.091865,0.094693,0.106219,0.101965,0.101300,0.097211,0.166872,0.115643,0.097758,...,0.117167,0.112685,0.159307,0.135460,0.150267,0.150057,0.130809,0.115740,0.137647,0.130409
4,0.156831,0.136079,0.138539,0.154710,0.150171,0.147664,0.141988,0.234334,0.150256,0.128399,...,0.171554,0.165990,0.191481,0.164326,0.181945,0.181117,0.160514,0.144114,0.165932,0.159441
5,0.191806,0.165784,0.172888,0.185544,0.184670,0.179056,0.172774,0.269306,0.205959,0.180736,...,0.211570,0.207750,0.223926,0.191774,0.217459,0.213872,0.189761,0.175509,0.197590,0.188914
6,0.151910,0.135060,0.137935,0.146178,0.146164,0.141231,0.137363,0.198211,0.200260,0.177503,...,0.210959,0.207012,0.194482,0.169995,0.190832,0.188341,0.173377,0.160079,0.173997,0.168929
7,0.198537,0.178650,0.181891,0.189349,0.188211,0.183258,0.180291,0.243414,0.184918,0.168680,...,0.177876,0.174240,0.260922,0.231061,0.253863,0.251180,0.235726,0.218665,0.230683,0.228329
8,0.156869,0.140667,0.143735,0.146398,0.146999,0.142869,0.140857,0.180463,0.183096,0.172287,...,0.199367,0.196415,0.171257,0.154462,0.168636,0.169781,0.159252,0.146634,0.154017,0.153548
9,0.202330,0.183625,0.187647,0.186966,0.188136,0.184292,0.179713,0.226676,0.200680,0.192393,...,0.225023,0.225495,0.244560,0.222121,0.239863,0.241639,0.233078,0.213097,0.221154,0.221033


## Exporting the datasets

In [48]:
# Configuration file for all tasks
task_config = {
    'folder': 'users/wivoliveira/ee_export',
    'scale': 10,
    #'region': geometry,
    'maxPixels': 1e9
    }

In [49]:
# VV data cube
task = ee.batch.Export.image.toDrive(vv.toBands(), str('VV'), **task_config)
task.start()

In [ ]:
# VH data cube
task2 = ee.batch.Export.image.toDrive(vh.toBands(), str('VH'), **task_config)
task2.start()

In [ ]:
# Ratio data cube
task3 = ee.batch.Export.image.toDrive(ratio.toBands(), str('Ratio'), **task_config)
task3.start()

In [ ]:
# RGI data cube
task4 = ee.batch.Export.image.toDrive(RGI.toBands(), str('RGI'), **task_config)
task4.start()

In [ ]:
# NL data cube
task5 = ee.batch.Export.image.toDrive(NL.toBands(), str('NL'), **task_config)
task5.start()